<a href="https://colab.research.google.com/github/Bonum/HTREC2022/blob/main/HTREC2022_SuperPeitho_WordSplitter_%5B12%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -U transformers
!pip install -U flair

In [ ]:
!git clone https://github.com/pranaydeeps/Ancient-Greek-BERT.git
%cd Ancient-Greek-BERT
# In case git lfs starts failing, because GitHub repository have bandwidth limit
# You can manually download from google drive link hosted by me as an alternative, unzip and save as final-model.pt
!git lfs pull --include "final-model.pt"

Cloning into 'Ancient-Greek-BERT'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 14), reused 18 (delta 1), pack-reused 0
Unpacking objects: 100% (50/50), done.
/content/Ancient-Greek-BERT
Git LFS: (1 of 1 files) 447.99 MB / 447.99 MB


In [ ]:
!mkdir ../LM/
!git clone https://huggingface.co/pranaydeeps/Ancient-Greek-BERT ../LM/SuperPeitho-v1

Cloning into '../LM/SuperPeitho-v1'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 37 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [ ]:
from flair.models import SequenceTagger
tagger = SequenceTagger.load("SuperPeitho-FLAIR-v2/final-model.pt")

2022-06-26 17:13:34,229 loading file SuperPeitho-FLAIR-v2/final-model.pt
2022-06-26 17:13:40,673 SequenceTagger predicts: Dictionary with 1030 tags: <unk>, O, a-p---na-, v2spma---, u--------, d--------, v-papamn-, r--------, l-s---ma-, n-s---ma-, v3siie---, l-s---nn-, l-s---fg-, n-s---fg-, l-s---mg-, n-s---mg-, v3ppia---, i--------, n-s---mn-, v3saia---, p-p---fd-, v-sppamn-, a-s---mn-, n-p---mg-, c--------, v3saoa---, p-s---mn-, l-s---mn-, v3siia---, v-sapamg-, b--------, p-s---cg-, p-s---fd-, l-p---mg-, a-p---mg-, a-s---ma-, v-sppamg-, v3spia---, a-p---ng-, n-p---ng-, _, v3piie---, l-p---md-, a-p---md-, v-pppamn-, p-p---ma-, l-s---fa-, n-s---fa-, n-p---na-, v3paia---


In [ ]:
from flair.data import Sentence

In [ ]:
%%capture
!pip install aicrowd-cli
%load_ext aicrowd.magic

In [ ]:
%aicrowd login


Please login here: https://api.aicrowd.com/auth/Kfu4xBmueJfmFLgoYICqzBG7NPk7JXBh5fMRmPtaUhE
API Key valid
Gitlab access token valid
Saved details successfully!


In [ ]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c htrec-2022 -o data

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

In [ ]:
%%capture
!pip install pywer
import pywer
import pandas as pd
import numpy as np
import os

In [ ]:
%%capture
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("pranaydeeps/Ancient-Greek-BERT")
model = AutoModel.from_pretrained("pranaydeeps/Ancient-Greek-BERT")

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
print(f"{train_df.shape[0]} train and {test_df.shape[0]} instances")
#train_df

1875 train and 338 instances


In [ ]:
#test_df

# Word Splitter

In [ ]:
#global cfg
cfg = {
    'min_split_len': 18,
    'st_off':     4,
    'end_off':    4,
    'min_score':  0.65, #0.5,
    'min_fsplit':  0.4,
    'sum_score':  1.1
  }
  
fcfg = {
    'min_split_len': 5,
    'st_off':     4, #2,
    'alt_off':    4, #2, 
    'end_off':    2,
    'min_split':  0.4,
    'min_max_split': 0.75, #  0.65,
    'sum_score':  1.0
  }
#ST_OFF=5 ; END_OFF=4 # -1.562	-9.980  cer: 99.2936048696175
# ST_OFF=6 ; END_OFF=6 #  -0.402	-2.265  cer: 0.6462763958818667
# ST_OFF=7 ; END_OFF=7 #  -0.164	-0.598  cer: 0.3456827233786729
# MIN_SCORE=0.4
# SUM_SCORE=1.1

ss=train_df.SYSTEM_TRANSCRIPTION[0]
hum=train_df.HUMAN_TRANSCRIPTION
wd=ss[0] #'ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει'
sswords=[w for w in ss.split() ]

def eval2words(w0, w1):
  sent=Sentence([w0, w1])
  tagger.predict(sent)
  #print(sent)
    
  t0=sent[0]
  t1=sent[1]
  s0,s1=t0.score, t1.score
  #print(f'scores: s0: {s0:.3f}, s1: {s1:.3f}, sum: {(s0+s1):.3f}')
  return s0, s1

In [ ]:
def morph_anal(sents):
  #v_n=['v', 'n']

  words=list(sents.split(" "))
  sumsc=0
  sent=Sentence(sents)
  tagger.predict(sent)
  #print(sent)
  ma=''
  for m in range(len(words)):
    #print(f'{m}, {sent[m].text} {sent[m].tag[0]} , {sent[m].score:0.3f}')
    sumsc+=sent[m].score
    ma+=sent[m].tag[0]
  return sumsc, ma

def sent2words(sent):
  words=list(sent.split(" "))
  print (words)
  w0=words[0]
  w1=words[1]
  return w0, w1

# for elem in sswords:
#   w0, w1 = sent2words(elem)
#   s0, s1 = eval2words(w0, w1)
sh='ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει'
sc,ma=morph_anal(sh)
sc,ma

(5.529698610305786, 'vndvbln')

In [ ]:
# for i in range(10):
#   print(hum[i])
#   sum_score(hum[i])

* l - article {του, της, τη, η}
* v - verb
* n -  noun
* b - preposifion {και, αλλά, 
* d - {καί,μή
* a - adjective {πολους, πονηρου }
* r  -  {προ, κατα, από, δία}
* c - { ώστε


In [ ]:
ss
sent=Sentence(ss)
tagger.predict(sent)
print(sent)
t=sent[2]
t.start_pos
t.tag[0]

Sentence: "ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει" → ["ἐγγενομεναπαδημησμεννωτες"/v-pppamn-, "ἀλλατῆε"/b--------, "κλησει"/n-s---fd-]


'n'

In [ ]:
def wsplit(word):
  MIN_WORD_LEN = cfg['st_off'] + cfg['end_off'] + 1
  if len(word) < MIN_WORD_LEN: return word
  maxsc=0; ms0=0; ms1=0; w1='*'; w2='*'
  mt0, mt1=None,None 
  #e_off = len(word)-cfg['end_off']
  for i in range(cfg['st_off'], len(word)-cfg['end_off']):
    w0=word[:i]
    w1=word[i:]
    #print(w1,  w2)3
    sent=Sentence([w0,w1])
    tagger.predict(sent)
    #print(sent)
    
    t0=sent[0]
    t1=sent[1]
    s0,s1=t0.score, t1.score
    # s0,s1=eval2words(w0, w1)
    #print(f'scores: s0: {ms0:.3f}, s1: {ms1:.3f}')
    if min(s0, s1) > cfg['min_score'] and (s0 + s1 > cfg['sum_score']):
        if s0 + s1 > maxsc:
            ms0=s0; ms1=s1
            mt0=t0; mt1=t1
            maxsc = s0 + s1
            mi = i
            #print(f'max_i {mi}, Scores: {s0:.3f}, {s1:.3f} Maxscore: {maxsc:.3f}')
            #print(f'Temp Found: {mt0}, {mt1}, {ms0:.3f}, {ms1:.3f}')
    
    #print(f'Found for {i} : {mt0}, {mt1}, {ms0:.3f}, {ms1:.3f}')
    #print(f'Iteration: {i}')

  #print(f'Final : {mt0}, {mt1}, {ms0:.3f}, {ms1:.3f}, max_score {maxsc:.3f}')
  if ms0==0 : return word
  else: return word[:mi]+' '+word[mi:]


In [ ]:
# sswords[1]
# split_pass2, _ = altern_splits(sswords[1], -1)
# split_pass2

In [ ]:
ss=train_df.SYSTEM_TRANSCRIPTION[0]
# #hum=train_df.HUMAN_TRANSCRIPTION[0]
# ss #'ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει'
sswords=[w for w in ss.split() ]
wd=sswords[0]
# split_pass1, _ = altern_splits(wd, -1)
# split_pass1

# From PyCharm

In [ ]:
# wwd='ἐγγενομεναπαδημησμεννωτες'
# #wwd='abcdefghijklmn'
# split_pass1, _ = altern_splits(wwd, -1)
# split_pass1


In [ ]:
fcfg111 = {
    'min_split_len': 5,
    'st_off': 2, #2,
    'alt_off': 2, #2,
    'end_off': 2, #2,
    'min_split': 0.4,
    'min_max_split': 0.75, #0.65,
    'sum_score': 1.0
}

def forced_split(word, s_off):
  MIN_WORD_LEN = fcfg['st_off'] + fcfg['end_off'] + 1
  #MIN_WORD_LEN = fcfg['st_off'] # + fcfg['end_off'] + 1

  if len(word) < MIN_WORD_LEN: 
        print('Min split! ',word)
        return word, True

  e_off = len(word)-fcfg['end_off']+1
  bSplitted = False
  for i in range(s_off, e_off):
    w0=word[:i]
    w1=word[i:]
    s0,s1=eval2words(w0, w1)
    #print(w0,w1)
    #print(f'{i}: Scores: {s0:.3f}, {s1:.3f}')
    #if min(s0, s1) > fcfg['min_split'] :
    if max(s0, s1) > fcfg['min_max_split'] :

      bSplitted = True
      return word[:i]+' '+word[i:], False

  if not bSplitted :
    return word, True

def altern_splits(word, iSortOrder):
  # iSortOrder: 1 : increasing, -1 : decreasing
  split_variants = []
  unique_list = []
  #print(len(word))
  for s_off in range(fcfg['st_off'], len(word)-fcfg['end_off'], fcfg['alt_off']):
    split_variant, bOneWord = forced_split(word, s_off)
    #print( 'offs,forced word, b: ', s_off, split_variant,bOneWord)
    if bOneWord: continue # return  split_variant, True
    split_variants.append(split_variant)
    list_set = set(split_variants)
    # convert the set to the list
    unique_list = (list(list_set))
    unique_list=sorted(unique_list, key=lambda x: (iSortOrder * len(x), x))
    unique_list
  return unique_list, False

#sswords=[w for w in ss.split() ]
#wd=wwd
#global split_pass1
#split_pass1, _ = altern_splits(wd, -1)
#print(split_pass1)

# Expand the list with the new one
#global split_full
#split_full = split_pass1.copy()

In [ ]:
#for s in split_full: print(s)
#print(split_full)    
def pass2(split_full, i, j):
    sswords = [w for w in split_pass1[i].split()]
    #print('-------> wd1 ', sswords[1])
    split_pass2, oneWord = altern_splits(sswords[1], 1)
    split_pass2 = list(reversed(split_pass2))
    print('=====> i,j,pas2',i,j) #,split_pass2)
    if split_pass2==[] or oneWord: return split_full, j
    row = split_pass1[i]
    w1, w2 = sent2words(row)
    for k in range(len(split_pass2)):
      pw1, pw2=sent2words(split_pass2[k])
      #print(row, w1, w2, pw1, pw2)
      new_row =' '.join([w1, pw1, pw2])
      # expanding
      if k == 0:
        split_full[i+j] = new_row
      else:
        split_full = split_full[0:i+j] + [new_row] + split_full[i+j:]

    j+=k
    return split_full, j


def big_word_splitt(wd):

  global split_pass1, split_full
  split_pass1, oneWord = altern_splits(wd, -1)
  if oneWord: return wd
  split_full = split_pass1.copy()

  j=0
  split_full_ini= split_full.copy()

  for i in range(len(split_full_ini)):
    split_full, j = pass2(split_full, i, j)

  max_scm=0; max_indm=0
  max_sc=0; max_ind=0
  for m,sents in enumerate(split_full): 
    sc,ma=morph_anal(sents)
    #print(sc,ma)
    if sc > max_scm and 'n' in ma : #and 'v' in ma: 
      max_scm=sc; max_indm=m
    elif sc > max_sc: 
      max_sc=sc; max_ind=m
  if max_scm==0: max_scm=max_sc; max_indm=max_ind  
  print('==========>', max_scm, max_indm)
  return split_full[max_indm]
    
#wwd=big_word_splitt(wwd)
#wwd
#print(split_full)

In [ ]:
def big_word_splitter(sent):

  words=[ w for w in sent.split() ]
  bigW = False
  for j,w in enumerate(words):
      if len(w) >= cfg['min_split_len'] : 
        print('bigw j,len',j,len(w))
        bigW = True
        bigWord = w
        print(w)
        break
  if not bigW: return sent

  print(sent)
  ext_list=[wsplit(w) for w in words]
  ext_list=' '.join(ext_list)
  words=list(ext_list.split(" "))

  #wd=wsplit(bigWord)
  wd=big_word_splitt(bigWord)

  #print("Substring found at index:", j)

  m=sent.find(bigWord)
  if m==0: st1=''
  else : st1=sent[0:m]
  st2=sent[m:]
  #print('Partial: st1 ', st1)
  #print('Partial: st2 ', '*'+st2+'*')
  
  k1=st2.find(' ')
  k=m+len(bigWord)
  print(k, k1, len(sent))
  if k < len(sent) : st3=sent[k:]
  else : st3 = ''

  print(st1+'*'+wd+'*'+st3)
  return st1 +wd +st3

i=213 #18
#for i in range(1,30):
#  print(i)
big_word_splitter(train_df.SYSTEM_TRANSCRIPTION[i])

bigw j,len 5 19
τοκαθεμακαιτοςκοςμο
ιαι τοῖ εμητις κοις καὶ τοκαθεμακαιτοςκοςμο
=====> i,j,pas2 0 0
['τοκα', 'θεμακαιτοςκοςμο']
['θεμα', 'καιτοςκοςμο']
=====> i,j,pas2 1 0
['τοκαθεμακα', 'ιτοςκοςμο']
['ιτοςκο', 'ςμο']
=====> i,j,pas2 2 0
==========> 1.5805107355117798 0
43 -1 43
ιαι τοῖ εμητις κοις καὶ *τοκα θεμα καιτοςκοςμο*


'ιαι τοῖ εμητις κοις καὶ τοκα θεμα καιτοςκοςμο'

# One split only

In [ ]:
# ext_list=[wsplit(w) for w in words]
# print('>>> no join extended list >>>>>>>>: ', ext_list)
# ext_list=' '.join(ext_list)
# print('>>>> join extended string >>>>>>>>: ', ext_list)

In [ ]:
def split_iter(spl_sent):
  MIN_WORD_LEN = cfg['st_off'] + cfg['end_off'] + 1
  #print(MIN_WORD_LEN)
  sent = spl_sent
  words=[ w for w in sent.split() ]
  
  #print(f'INITIAL: Number of  words : {len(words)}')
  while True:
    bigW = False
    for w in words:
      if len(w) >= MIN_WORD_LEN : bigW = True
    if not bigW: break

    num_words=len(words)
    ext_list=[wsplit(w) for w in words]
    ext_list=' '.join(ext_list)
    words=list(ext_list.split(" "))
    print ("Intermediate: ", words)
    print(f'Num new words : {len(words)}; Num old list words: {num_words} ')
    if len(words) <= num_words: break # No more extensions found

  return ' '.join(words)

# ws_str = split_iter(train_df.SYSTEM_TRANSCRIPTION[0])
# ws_str 

In [ ]:
def prn_res(i, ws_str, df):
  print('SY ', df.SYSTEM_TRANSCRIPTION[i])
  print('WS ', ws_str)
  if df is train_df: print('HU ', df.HUMAN_TRANSCRIPTION[i])
  

In [ ]:
for i in range(0,1):
  ws_str = split_iter(train_df.SYSTEM_TRANSCRIPTION[i])
  #ws_str=' '.join(final_list)
  prn_res(i, ws_str, train_df)

Intermediate:  ['ἐγγενομεν', 'απαδημησμεννωτες', 'ἀλλατῆε', 'κλησει']
Num new words : 4; Num old list words: 3 
Intermediate:  ['ἐγγενομεν', 'απαδημησμεννωτες', 'ἀλλατῆε', 'κλησει']
Num new words : 4; Num old list words: 4 
SY  ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει
WS  ἐγγενομεν απαδημησμεννωτες ἀλλατῆε κλησει
HU  ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει


# Test different parameters

# Pattern  matching

In [ ]:
#PATT="τῶν" #PATT="των"
#PATT="της" # -0.033	-0.192 189559 WORD_SCORE=0.998
#PATT="τῆς" # 0.032

# PATT="τῆv" # None
# PATT="τηv" #None

#PATT="του" 
#PATT='τού' 
PATT='τοκαθεμακαιτοςκοςμο' #'τις'
#PATT="τοῦ"
#PATT='ἐπι' 
#PATT='ἀλλ᾽' #PATT='ἀλλ '
#PATT=' ι ' PATT=' ἀ 'PATT='ἀλλο' ἀλλοῖσιτουτ᾽
#PATT='τουτ' #PATT='τασ' => 0.0
#PATT= ' ἐ ' #δ ' #' ί ' #' ῶ ' #ω '#'κ  ρ '  #' β '#' ο '#' α ' #' ι ' # ' ε ' #'δὲ' #'τής' #' πολλα' PATT='πολλαίμν'

#train_df[train_df.HUMAN_TRANSCRIPTION.str.contains( PATT )][:50]
train_df[train_df.SYSTEM_TRANSCRIPTION.str.contains( PATT )][:50]

#test_df[test_df.SYSTEM_TRANSCRIPTION.str.contains( PATT )].SYSTEM_TRANSCRIPTION



,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM,B1,B1_CER,CER
213,και τους μηνισκους και το καθεμα και τον κοσμον,ιαι τοῖ εμητις κοις καὶ τοκαθεμακαιτοςκοςμο,11,8 Bodleian-Library-MS-Barocci-102_00164_fol-78...,5,ιαι τοῖ εμητις κοις καὶ τοκαθεμακαιτοςκοςμο,34.042553,34.042553


In [ ]:
# cfg = {
#     'tis' : 0.998, # lb 0.032
#     'ths' : 0.800, # lb -0.013  ; 0.909  # lb -0.033
#     'polla' : 0.900
# }

patts=['της', 'τῆς', 'τής', 'πολλα', 'ἐπι', 'ἀλλ', 'ἀλλῶ', 'ὑπερ', 'τού', 'τοῖς', 'αυτων', 'τις']
maxsc=[0.998, 0.800, 0.998, 0.9, 0.998, 0.998, 0.995, 0.998, 0.998, 0.999, 0.998, 0.999]

pa='τής'
maxsc[patts.index(f'{pa}')]

0.998

In [ ]:
#WORD_SCORE= 0.909 #0.850 cerr 0.0147 #0.909 cerr 0.0176 #0.950 #0.998
WLEN_SPLIT = 15      # split above
WLEN_NOT_SPLIT = 3  # not split below: this + len(pat)
SPLIT_ABOVE = 15

def tsc(word):
  sent=Sentence([word])
  tagger.predict(sent)
  return sent[0].score

def wpat(word, pat):
  SL=len(pat)
  if len(word)==SL:return word
  #print('word score ', tsc(word))
  i = word.find(pat)
  #print("Substring found at index:", i)
  if i==0: w1=word[0:SL]; w2=word[SL:]
  else : w1=word[0:i]; w2=word[i:]
  #print(w1,w2)
  sent=Sentence([w1,w2])
  tagger.predict(sent)
  t1=sent[0]; t2=sent[1]
  s1,s2 = t1.score, t2.score
  print(f'{tsc(word):.3f} =>{s1:.3f}, {s2:.3f}')

  WSC=maxsc[patts.index(f'{pat}')]

  if max(s1,s2) > 0.8 * WSC  and len(word) > SPLIT_ABOVE: return w1+' '+w2
  # unremark for POLLA: if min(len(w1), len(w2)) <= WLEN_NOT_SPLIT : return word
  if tsc(word) > WSC : return word
  if tsc(word) > max(s1,s2) : return word
  else : return w1+' '+w2

def pattern(sent, pat):
  ii = sent.find(pat)
  if ii < 0 : return sent
  words=[ w for w in sent.split() ]
  for j,w in enumerate(words):
    iw = w.find(pat)
    if iw < 0: continue
    else : break

  if j==0: ispw = 0; s0='' # pat is the first word in sent, don't look for possible next ' '+w
  else : 
    ispw = sent.find(' '+w)
    if ispw < 0: ispw=0; s0=''
    else: ispw+=1; s0=sent[:ispw]
    
  ispe = sent.find(' ',ii)
  if ispe < 0: ispe=len(sent); s1=''
  else : s1=sent[ispe:]

  print(sent)
  #print('ispw ', ispw, 'ispe ',ispe)
  word=sent[ispw:ispe]
  #print('*'+word+'*')
  tword=wpat(word, pat) # process pattern in word
  print(s0+tword+s1)
  return s0+tword+s1



In [ ]:
def ths(sent):
  return pattern(sent, 'της')

def tiss(sent):
  return pattern(sent, 'τις')


def tis(sent):
  return pattern(sent, 'τῆς')

def polla(sent):
  return pattern(sent, 'πολλα')

def tiis(sent):
  return pattern(sent, 'τής')
ss=train_df.iloc[134].SYSTEM_TRANSCRIPTION

def All(sent):
  return pattern(sent, 'ἀλλ')

def epi(sent):
  return pattern(sent, 'ἐπι')
  
def allo(sent):
  return pattern(sent, 'ἀλλῶ')
  
def epi(sent):
  return pattern(sent, 'ἐπι')

def tou(sent):
  return pattern(sent, 'τού')
  
def uper(sent):
  return pattern(sent, 'ὑπερ')

def tois(sent):
  return pattern(sent, 'τοῖς')

def auton(sent):
  return pattern(sent, 'αυτων')
  
 
 
#ss=test_df.iloc[44].SYSTEM_TRANSCRIPTION
print(ss)
rtis=tiis(ss)
rtis
 

των ερευπτων ταμεν τής σάρκος παραπετα
των ερευπτων ταμεν τής σάρκος παραπετα
των ερευπτων ταμεν τής σάρκος παραπετα


'των ερευπτων ταμεν τής σάρκος παραπετα'

In [ ]:
#test_df[test_df.SYSTEM_TRANSCRIPTION.str.contains("τῆς")] 


# Tests and CERR

*  WS: ST_OFF=5 ; END_OFF=4 # -1.562	-9.980  cer: 99.2936
*  WS: ST_OFF=6 ; END_OFF=6 #  -0.402	-2.265  cer: 0.64627
*  WS: ST_OFF=7 ; END_OFF=7 #  -0.164	-0.598  cer: 0.34568
*  S1: > > >  -0.152	-0.565 cer (R0,S1):    0.38325
* 189440:  lmr + dot: -0.017	-0.238 [ cer(R0,lmr) 0.0375, cer(R0, dot) 0.0901, cer(R0, lmr+dot)  0.1277 
* 189536: -0.008	0.000  serr 0.14, 0.0375
* 189549	 TIS (WC=0.998)	0.032	-0.022
* 189604: ALL 	-0.013	0.008
* 189610: THS (WC=0.8) 	-0.013	-0.075
* 189671: THS (WC=0.55)	-0.013	-0.009
* 189674! TIS (WC=0.93)	0.032	-0.022
* 189704: TAS	0.000	0.000
* 189710: POLLA	0.000	0.000
* 189914	TIIS (WC=0.998)	-0.006	0.000
* 189926!	EPS	 0.006	0.101
* 189928	IOT	-0.007	0.000
* 190011	OMG	-0.038	-0.059
* 190019	DLT	-0.017	-0.042
* 190103	BWS (4,4,0.5) 	-0.032	0.025
* 190104	BWS (4,4,0.65) 	-0.051	-0.049
* 190286	BWS (4,5,0.8) 	-0.085	-0.138
* 190290	 EPI	-0.003	-0.037 cer:    0.010065280533747448 CERR  -0.003862656493202014
* 190291	AALL -0.014	-0.094 cer:    0.05032640266873724 CERR  -0.001385179750976074
* 190293!  UPER 0.001	-0.099   cer:    0.00862738331464067 CERR  -0.004430143567398466
* 190426	 TOU	0.000	0.000
* 190427	 TOIS -0.012	-0.037
* 190430	 AUTON -0.014	-0.074
* 190431   EPS+UPER: 0.007	0.003
* 190436   TISS: -0.015	-0.020


 cfg['st_off'] = 4; cfg['end_off']=5; cfg['min_split'] = 0.8

* 190672	Ws	'vn' -0.080	-0.173
* 190676	BWS 'n'	 -0.072	-0.173
* 190716  BWS	-0.084	-0.088  cfg['min_max'] = 0.65
* 190717	Bws	-0.063	-0.123   -'- 0.75
*    719    Bws -0.108	-0.173 -'- cfg['st_off']='endoff' = 4

In [ ]:
B0 = train_df.SYSTEM_TRANSCRIPTION


#B0
R0 = test_df.SYSTEM_TRANSCRIPTION
R0[320:]
#R3 = test_df.SYSTEM_TRANSCRIPTION.apply(split_iter)

320                      στικῆ κ ἀτελεύτητος εξις, καὶ ς
321                εαπειν πασης ουσίας κιίςδυναμεω  ἐνερ
322    γειας α᾽ρχής τε κ μεσοτητος καὶ τἣέλους πὩοιητικη
323        η ἀρχῆ κ μεσοτης και νῦνκοὐκ το τελος τῶν χρό
324        νω Ξανθία. διαιρετῶν εισι γνωρίσματα. ειποι δ
325            αν τις αλκηθεύων καὶ τῶνΒἢ εν αἰῶνι συνορ
326    ωμενων ο μέν αὐτοῖς γὰρ χρόνος μετρουμενην ἔχω...
327            ἐκείνου κίνησιν ἀριθμω περιγραφεται. ὁ αι
328             ῶᾦν δὲ συνεπινοουμενην ὃἔχων τῆ ὑπαἙρξει
329       Ἕκτωρ τ᾽ ἀδελφὸς οὑμὸς εὐτύχει δορί, μεταβολὴν
330                   καλῶς παρ᾽ ἀῃδρὶ Θρκνὶ πατρῴῳ ξένῳ
331               τροφαῖσιν ὥς τιςπτόρθος ηὐξόμην τάλας·
332                     ἐπεὶ Τροία θ᾽ Ἕκτορός τ᾽ ἀπώλετο
333                       ψυχὴ πακρῴα θ᾽ κστία ἑατεστάφη
334                         τε βωμῷ πρὸς θεοδμήτῳ πίτνει
335             σφαγεὶς Ἀχιλλέως παὺιδὸς ἐἴἥκ μιαιφόνου,
336               κτείνει ωε χρυμοῦ τὸν ταλαίσπρον χάριν
337               ξένος πατρῷος

In [ ]:
def cerr():
  #!! Assume the prediction for training set is in B1 !!!
  train_df["B1"] = B1 #train_df.SYSTEM_TRANSCRIPTION.apply(split_iter)  #lmr)

  # Calculate CER for baseline predictions
  train_df["B1_CER"] = train_df.apply(lambda row: pywer.cer([row.HUMAN_TRANSCRIPTION], [row.B1]), axis=1)
  #print(f"B1 CER: {train_df.B1_CER.mean()}")

  # Computing the character error *reduction* rate (CERR)
  train_df["CER"] = train_df.apply(lambda row: pywer.cer([row.HUMAN_TRANSCRIPTION], [row.SYSTEM_TRANSCRIPTION]), axis=1)
  ret_cerr = (train_df.CER - train_df.B1_CER).mean()
  print(f"B1 CERR: {ret_cerr}")
  return ret_cerr

In [ ]:
# import time
# start_time = time.time()
# end_time = time.time()
# time_elapsed = (end_time - start_time)
# print(time_elapsed)

In [ ]:
# #%%capture
# split_offs = pd.DataFrame(columns=["st_off", "end_off", "min_split", "time", "CERR"])
# # MIN_WORD_LEN = cfg['st_off'] + cfg['end_off'] + 1
# # 4,4, 0.5-> 0.65
# for min_sc in [3.5] : # np.linspace(0.35, 0.8, 4):
#   for st_off in range(3,4): #(3,6):
#     for end_off in range(3,4):
#       print(f'####################################################################### st_off: {st_off}, end_off: {end_off}, min_sc: {min_sc:.3f}')
#       start_time = time.time()
#       cfg['min_split'] = min_sc
#       cfg['st_off'] = st_off; cfg['end_off']=end_off

#       B1=B0.apply(split_big_word)
#       res=cerr()
#       end_time = time.time()
#       t_elapsed = (end_time - start_time)
#       new_row = {'st_off':st_off, 'end_off':end_off, 'min_split':min_sc, 'time':t_elapsed, 'CERR':res }
#       split_offs = split_offs.append(new_row, ignore_index=True)

# print('time elapsed ', t_elapsed) # 12 min
# split_offs

In [ ]:
#split_offs = pd.DataFrame(columns=["st_off", "end_off", "min_split", "CERR"])
#split_offs

In [ ]:
# split_cerr = pd.DataFrame(columns=["MinScore","SumScore","CERR"])
# new_row = {'MinScore':0.5, 'SumScore':1.23, 'CERR':0.98 }
# split_cerr = split_cerr.append(new_row, ignore_index=True)
# split_cerr

In [ ]:
# ST_OFF=7 ; END_OFF=7 
# MIN_WORD_LEN=ST_OFF+END_OFF+1
# MIN_SCORE=0.4
# SUM_SCORE=1.1
# #split_cerr = pd.DataFrame(columns=["MinScore","SumScore","CERR"])
# split_offs = pd.DataFrame(columns=["ST_OFF","END_OFF","CERR"])

# # 0.3900	1.133333 ===>	0.014362  
# # for min_sc in np.linspace(0.39, 0.6, 5):
# #   for sum_sc in np.linspace(1.0, 1.4, 4):
# for st_off in range(3,9):
#   for end_off in range(3,9):
#     ST_OFF = st_off; END_OFF=end_off
#     B1=B0.apply(split_iter)
#     res=cerr()
#     new_row = {'ST_OFF':st_off, 'END_OFF':end_off, 'CERR':res }
#     split_offs = split_offs.append(new_row, ignore_index=True)


In [ ]:
# split_offs = pd.DataFrame(columns=["ST_OFF","END_OFF","CERR"])
# # 3.0	3.0	0.014362
# # 7.0	7.0	0.014362
# for st_off in range(9,10):
#   for end_off in range(9,10):
#     cfg['st_off'] = st_off; cfg['end_off']=end_off
#     B1=B0.apply(split_iter)
#     res=cerr()
#     new_row = {'ST_OFF':st_off, 'END_OFF':end_off, 'CERR':res }
#     split_offs = split_offs.append(new_row, ignore_index=True)
# split_offs

In [ ]:
# ST_OFF=8; END_OFF=8; 
# B1=B0.apply(split_iter)
# res=cerr()

In [ ]:
# cerr_res=[]
# cerr_df = pd.DataFrame(columns=['WordScore','CERR'])
# pa='τής'
# for wsc in np.linspace(0.6, 0.9999, 10):
#   maxsc[patts.index(f'{pa}')] = wsc
#   B1=B0.apply(tiis)
#   res=cerr()
#   new_row = {'WordScore':wsc, 'CERR':res}
#   cerr_df = cerr_df.append(new_row, ignore_index=True)

#   print(">>>>>>>>>>>>>>>> ", wsc, res, ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>" )

# cerr_df

# Save to Google Drive or Download

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')
# path = '/content/drive/My Drive/split_offs.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   split_offs.to_csv(f)

# # from google.colab import files

# # tis_cerr.to_csv('tis_cerr.csv', encoding = 'utf-8-sig') 
# # files.download('tis_cerr.csv')

In [ ]:
# # Use cfg['ths'] = 0.55 || cfg['tis'] = 0.93 
# cfg['tis'] = 0.93 

# LMing

In [ ]:
!git clone https://github.com/ipavlopoulos/lm.git
from lm.markov.models import LM
lm = LM(gram="CHAR").train(train_df.HUMAN_TRANSCRIPTION.sum()); #cslm.generate_text()

Cloning into 'lm'...
remote: Enumerating objects: 501, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 501 (delta 14), reused 11 (delta 11), pack-reused 482
Receiving objects: 100% (501/501), 114.89 KiB | 9.57 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [ ]:
def lmr(text, word=" ς ", replacements=["ς ", " "], lm=lm):
    scores = []
    for the_candidate in replacements:
        scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
    text_out = text.replace(word, replacements[scores.index(min(scores))])
    return text_out
    
lmr("ως ω ς αὐτοῦ ον πος τοῦ χυ καιρήε,")

'ως ως αὐτοῦ ον πος τοῦ χυ καιρήε,'

In [ ]:
# def dot(text, word="·", replacements=["", ","], lm=lm):  
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out
    
# dot("ως λογον· ")

In [ ]:
def eps(text, word=" ε ", replacements=[' ', ' ε', 'ε '], lm=lm):  
    scores = []
    for the_candidate in replacements:
        scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
    text_out = text.replace(word, replacements[scores.index(min(scores))])
    return text_out

print(train_df.iloc[1125].SYSTEM_TRANSCRIPTION)
eps(train_df.iloc[1125].SYSTEM_TRANSCRIPTION)

επαιρομέν ε κατα τῆς ἵνωσεως του


'επαιρομέν κατα τῆς ἵνωσεως του'

In [ ]:
# def Tis(text, word="τής ", replacements=[" τής "], lm=lm):  
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out
    
# Tis(all(train_df.iloc[183].SYSTEM_TRANSCRIPTION))


In [ ]:
# def all(text, word='ἀλλ ', replacements=['ἀλλ᾽' , 'ἀλλ'], lm=lm):
# #def all(text, word="ἀλλ ", replacements=["ἀλλ"], lm=lm):
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out
    
# all(train_df.iloc[948].SYSTEM_TRANSCRIPTION)



In [ ]:
# #PATT="τού" τοῦ
# #PATT='ἀλλ ' 
# def tou(text, word="τοῦ ", replacements=["τού", "τοῦ"], lm=lm):
# #def all(text, word="ἀλλ ", replacements=["ἀλλ"], lm=lm):
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out
    
# tou(train_df.iloc[1590].SYSTEM_TRANSCRIPTION)



In [ ]:
# def iot(text, word=" ι ", replacements=[' ', ' ι', 'ι '], lm=lm):  
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out

# print(train_df.iloc[649].SYSTEM_TRANSCRIPTION)
# iot(train_df.iloc[649].SYSTEM_TRANSCRIPTION)

In [ ]:
# def omg(text, word=" ω ", replacements=[' ', ' ὡ', 'ὡ ', ' ω', 'ω '], lm=lm):  
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out

# sent = train_df.iloc[1156].SYSTEM_TRANSCRIPTION
# print(sent)
# omg(sent)

In [ ]:
# def dlt(text, word=' δ ', replacements=[' ', ' δ', 'δ ', ' δ᾽ '], lm=lm):  
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out

# i=487
# sent = train_df.iloc[i].SYSTEM_TRANSCRIPTION
# print(sent)
# print(train_df.iloc[i].HUMAN_TRANSCRIPTION)
# dlt(sent)

In [ ]:
# def tas(text, word=" τασ", replacements=[" τας "], lm=lm):
#     scores = []
#     for the_candidate in replacements:
#         scores.append(lm.cross_entropy(text.replace(word, the_candidate)))
#     text_out = text.replace(word, replacements[scores.index(min(scores))])
#     return text_out
    
# tas(train_df.iloc[100].SYSTEM_TRANSCRIPTION)



 **Predictions**

In [ ]:
B1=B0.apply(eps)
print('cer:   ', pywer.cer(R0,B1))

cer:    89.14856842263471


In [ ]:
B1=R0.apply(lmr)
print('cer:   ', pywer.cer(R0,B1))

cer:    0.037574209062899225


In [ ]:
# ALL=R0.apply(all)
# print('cer:   ', pywer.cer(R0,ALL))

In [ ]:
# D=S1.apply(dot)
# print('cer:   ', pywer.cer(R0,D))

In [ ]:
#TIS=R0.apply(tis)
#print('cer:   ', pywer.cer(R0,TIS))

In [ ]:
# POLLA=R0.apply(polla)
# print('cer:   ', pywer.cer(R0,POLLA))

In [ ]:
EPS=R0.apply(eps)
print('cer:   ', pywer.cer(R0,EPS))
# B1=B0.apply(eps)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

cer:    0.022544525437739535


In [ ]:
# TISS=R0.apply(tiss)
# print('cer:   ', pywer.cer(R0, TISS))
# B1=B0.apply(tiss)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

In [ ]:
# AUTON=R0.apply(auton)
# print('cer:   ', pywer.cer(R0, AUTON))
# B1=B0.apply(auton)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

In [ ]:
# AALL=R0.apply(All)
# print('cer:   ', pywer.cer(R0,AALL))
# B1=B0.apply(All)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

In [ ]:
# #UPER=EPS.apply(uper)
# UPER=R0.apply(uper)
# print('cer:   ', pywer.cer(R0,UPER))
# B1=B0.apply(uper)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

In [ ]:
# EPI=R0.apply(epi)
# print('cer:   ', pywer.cer(R0,EPI))
# # B1=B0.apply(epi)
# # print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())

In [ ]:
# D0=R0.apply(dot)
# print('cer:   ', pywer.cer(R0,D0))

In [ ]:
# B1=B0.apply(omg)
# print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())
# OMG=R0.apply(omg)
# print('cer:   ', pywer.cer(R0,OMG))

In [ ]:
B1=B0.apply(big_word_splitter)
print('cer:   ', pywer.cer(B0,B1), 'CERR ', cerr())



In [ ]:
BWS=R0.apply(big_word_splitter)
print('cer:   ', pywer.cer(R0,BWS))

bigw j,len 2 19
συνευςδιξοεανδνυπορ
ὑπρ τη συνευςδιξοεανδνυπορ ποοδυπρας
=====> i,j,pas2 0 0
['συνευςδιξο', 'εανδνυπορ']
['εανδ', 'νυπορ']
=====> i,j,pas2 1 0
==========> 1.9300822764635086 0
26 19 36
ὑπρ τη *συνευςδιξο εανδ νυπορ* ποοδυπρας
bigw j,len 0 21
καἀκόδετπτεισκεικύφις
καἀκόδετπτεισκεικύφις κακλα καντκρας
=====> i,j,pas2 0 0
['καἀκ', 'όδετπτεισκεικύφις']
['όδετπτεισκεικ', 'ύφις']
['όδετπτεισκε', 'ικύφις']
['όδετ', 'πτεισκεικύφις']
=====> i,j,pas2 1 2
=====> i,j,pas2 2 2
==========> 1.6179803311824799 1
21 21 36
*καἀκ όδετπτεισκε ικύφις* κακλα καντκρας
bigw j,len 1 21
ἀκολουθεδεύρότάσκευηφ
αὶπαις ἀκολουθεδεύρότάσκευηφ
=====> i,j,pas2 0 0
==========> 1.0805111825466156 0
28 -1 28
αὶπαις *ἀκολουθεδεύρότάσκευ ηφ*
bigw j,len 1 20
παλινοισειςταστρωμλς
ἀλλραμένος παλινοισειςταστρωμλς
=====> i,j,pas2 0 0
['παλι', 'νοισειςταστρωμλς']
['νοισειςτ', 'αστρωμλς']
['νοισ', 'ειςταστρωμλς']
=====> i,j,pas2 1 1
['παλινοισ', 'ειςταστρωμλς']
['ειςταστρ', 'ωμλς']
['ειςτ', 'αστρωμλς']
=====> i,j,p


# Submissions::


In [ ]:
submission = pd.DataFrame(zip(test_df.IMAGE_PATH, BWS), columns=["ImageID", "Transcriptions"])
#submission

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
#%aicrowd submission create -c htrec-2022 -f submission.csv

Output()

╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯

                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/htrec-2022/submissions/190719              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/htrec-2022/submissions?my_submissions=true │
│                  │                                                                               │
│      Leaderboard │ https://www.aicrowd.com/challenges/htrec-2022/leaderboards                    │
│                  │                                                                               │
│ Discussion forum │ https://discourse.aicrowd.com/c/htrec-2022                                    │
│                  │                                                                               │
│   Challenge page │ https://www.aicrowd.com/challenges/htrec-2022                                 │
└──────────────────┴───────────────────────────────────────────────────────────────────────────────┘

{'submission_id': 190719, 'created_at': '2022-06-26T19:12:30.109Z'}
